In [ ]:
#to have acces to IBM 
from qiskit_ibm_runtime import QiskitRuntimeService
from dotenv import load_dotenv
import os

load_dotenv(override=True)
token = os.environ['IBM_TOKEN']
service = QiskitRuntimeService(channel="ibm_quantum", token=token)

# Exercise 4: Quantum noise

In [ ]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler

shots = 500

qc = QuantumCircuit(2)  # 2 qubits

# Step 2: Create entanglement using Hadamard and CNOT gates
qc.h(0)  # Apply Hadamard gate to qubit 0
qc.cx(0, 1)  # Apply CNOT gate with qubit 0 as control and qubit 1 as target

# Step 3: Add measurement to the circuit
qc.measure_all()

# Display the quantum circuit
print("Quantum Circuit:")
display(qc.draw('mpl'))

Same circuit as before and we have the final state of the system 
$$
\psi = \frac{1}{\sqrt{2}} (\ \ket{00} + \ket{11} )\
$$

But this time to have it work properly on a real quantum computer we need to modify the circuit to only use the available gates
<br>
We call this transpiling a circuit so that this will be exactly the same state at the end as before but only using the availble gates. 

In [ ]:
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
optimisation_level = 1

#pass_manager = informations needed to transpile the circuit following the requirement of the machine
pass_manager = generate_preset_pass_manager(backend=backend, optimization_level=optimisation_level)

transpile_circuit = pass_manager.run(qc)
display(transpile_circuit.draw('mpl', idle_wires=False))

To not wait 40 minutes queue i have already run the circuit earlier so we will just call the results back form IBM

In [ ]:

# first time dooing it
# sampler = Sampler(mode=backend)
# job = sampler.run([transpile_circuit], shots=shots)

# take value back from IMBQ account
job = service.job('cwbqmg2ggr6g0088zv0g')


result = job.result()[0].data.meas.get_counts()
frequencies = {key: value / shots for key, value in result.items()}


# Step 6: Plot the histogram of frequencies
display(plot_histogram(frequencies, figsize=(7, 5)))

As we can see we don't have a perfect system as before
<br>
This is due to all the noise that will interfere with the real quantum componants of the quantum computer
<br>
In quantum physics if there is an observation the state will collaps to the given state so

for example if 
$$
\psi = \frac{1}{\sqrt{2}} (\ \ket{00} + \ket{11} )\
$$
if a particle or even a photon hits one of the qbits it will behave differnetly if it hits a qbit in $\ket{0}$ or $\ket{1}$
<br>
This effectivly counts as a "Observation" so instead of a superpositions of the 2 states what we will instead have is 
<br>
$$
\psi = \ket{00}\\
or\\
\psi = \ket{11}
$$